<a href="https://colab.research.google.com/github/leonbenenati/LH_CD_Leon/blob/main/LH_CD_Leon_Emiliano_Benenati_Previsao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem

* O objetivo é fazer um modelo para prever preço de aluguel

* Primeiro passo é escolher as colunas que serão usadas.
Foram usadas as colunas: bairro_group	bairro	room_type	minimo_noites	numero_de_reviews	calculado_host_listings_count	disponibilidade_365

As colunas id', 'nome', 'host_id',"host_name"são relacionadas a identificação. "latitude", "longitude" não é necessário porque será usada as colunas de distrito e bairros e id'. "reviews_por_mes" tem alta correlação com numero_de_reviews e "ultima_review" é data do último review

* Será divido a base de treino e teste, o modelo será criado com a base de treino e a base de teste será usada para testar, como o modelo não viu os dados durante o treinamento simula vida real com novos dados. Isso diminui a chance de ocorrer Overfitting ou underfitting


* Nas variavéis categóricas será utilizado o Target Encoder.O Target Encode consiste em substituir a categoria pela média do alvo, exemplo: na coluna bairro_group a categoria Manhattan será agrupada e será feita a média dos preços e será substituída pelo valor

* O metódo de Target Encoder será utilizado para não criar mais colunas, em bairro a coluna tem mais de 200 categorias, se fosse utilzar one hot encoder iria criar mais de 200 colunas a mais e poderia cair na maldição da dimensionalidade.

* Será feito um modelo base e um modelo com preços de até 500 dólares e será comparada a perfomace

* Será utilizado o LGBMRegressor. O modelo é rapido e da boas perfomaces, a desvantagem é que para ter a importância de cada váriavel não é tão fácil e direita como a regressão linear, mas é possível com o shap values

* As métricas de avaliação será o MSE e MAE, Erro Quadrático Médio(MSE) é uma métrica que penaliza erros por elevar ao quadrado. Erro Absoluto Médio(MAE) é uma métrica que é melhor para interpretar pois não é elevado ao quadrado

* Será comparada modelo base e modelo com preços até 500, o melhor será escolhido para fazer a tunagem de hiperparâmetros

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("/content/teste_indicium_precificacao.csv")
df.head()

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


In [ ]:
# dropando as variaveis que não serão usadas
df.drop(['id', 'nome', 'host_id',"host_name","latitude", "longitude", "ultima_review", "reviews_por_mes" ], axis = 1, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48894 entries, 0 to 48893
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   bairro_group                   48894 non-null  object
 1   bairro                         48894 non-null  object
 2   room_type                      48894 non-null  object
 3   price                          48894 non-null  int64 
 4   minimo_noites                  48894 non-null  int64 
 5   numero_de_reviews              48894 non-null  int64 
 6   calculado_host_listings_count  48894 non-null  int64 
 7   disponibilidade_365            48894 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 3.0+ MB


In [ ]:
df.head()

,bairro_group,bairro,room_type,price,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365
0,Manhattan,Midtown,Entire home/apt,225,1,45,2,355
1,Manhattan,Harlem,Private room,150,3,0,1,365
2,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,1,194
3,Manhattan,East Harlem,Entire home/apt,80,10,9,1,0
4,Manhattan,Murray Hill,Entire home/apt,200,3,74,1,129


In [ ]:
df.isnull().sum()

bairro_group                     0
bairro                           0
room_type                        0
price                            0
minimo_noites                    0
numero_de_reviews                0
calculado_host_listings_count    0
disponibilidade_365              0
dtype: int64

# Preparação dos dados

In [ ]:
X = df.drop(["price"], axis = 1)
y = df['price']


In [ ]:
X

,bairro_group,bairro,room_type,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365
0,Manhattan,Midtown,Entire home/apt,1,45,2,355
1,Manhattan,Harlem,Private room,3,0,1,365
2,Brooklyn,Clinton Hill,Entire home/apt,1,270,1,194
3,Manhattan,East Harlem,Entire home/apt,10,9,1,0
4,Manhattan,Murray Hill,Entire home/apt,3,74,1,129
...,...,...,...,...,...,...,...
48889,Brooklyn,Bedford-Stuyvesant,Private room,2,0,2,9
48890,Brooklyn,Bushwick,Private room,4,0,2,36
48891,Manhattan,Harlem,Entire home/apt,10,0,1,27
48892,Manhattan,Hell's Kitchen,Shared room,1,0,6,2


In [ ]:
#Divisão de treino e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=30)

In [ ]:
X_train

,bairro_group,bairro,room_type,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365
45708,Brooklyn,Sunset Park,Private room,1,7,2,50
33440,Staten Island,Concord,Private room,4,11,7,337
21213,Brooklyn,Bushwick,Private room,3,6,1,0
20048,Manhattan,Inwood,Entire home/apt,5,39,1,12
20963,Queens,Jamaica,Private room,1,10,1,175
...,...,...,...,...,...,...,...
33268,Brooklyn,Williamsburg,Private room,30,3,3,19
44845,Manhattan,Theater District,Private room,2,10,1,24
48045,Manhattan,Murray Hill,Entire home/apt,1,0,1,176
4517,Manhattan,Upper East Side,Entire home/apt,5,22,1,7


# Target Encoder

In [ ]:
from category_encoders import TargetEncoder

In [ ]:
#Transofrmado variáveis categóricas
TE_encoder = TargetEncoder()
train_te = TE_encoder.fit_transform(X_train[["bairro_group","bairro","room_type"]], y_train)
test_te = TE_encoder.transform(X_test[["bairro_group","bairro","room_type"]])

train_te.head()

,bairro_group,bairro,room_type
45708,124.577136,103.590747,90.114488
33440,118.522388,95.635395,90.114488
21213,124.577136,84.869048,90.114488
20048,196.847845,90.224049,211.324773
20963,99.649351,95.077868,90.114488


In [ ]:
#Juntando os dados depois da transoformação na base original
X_train=X_train.merge(train_te, left_index=True,right_index=True)

In [ ]:
X_train

,bairro_group_x,bairro_x,room_type_x,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365,bairro_group_y,bairro_y,room_type_y
45708,Brooklyn,Sunset Park,Private room,1,7,2,50,124.577136,103.590747,90.114488
33440,Staten Island,Concord,Private room,4,11,7,337,118.522388,95.635395,90.114488
21213,Brooklyn,Bushwick,Private room,3,6,1,0,124.577136,84.869048,90.114488
20048,Manhattan,Inwood,Entire home/apt,5,39,1,12,196.847845,90.224049,211.324773
20963,Queens,Jamaica,Private room,1,10,1,175,99.649351,95.077868,90.114488
...,...,...,...,...,...,...,...,...,...,...
33268,Brooklyn,Williamsburg,Private room,30,3,3,19,124.577136,143.378788,90.114488
44845,Manhattan,Theater District,Private room,2,10,1,24,196.847845,243.422109,90.114488
48045,Manhattan,Murray Hill,Entire home/apt,1,0,1,176,196.847845,214.223529,211.324773
4517,Manhattan,Upper East Side,Entire home/apt,5,22,1,7,196.847845,184.584786,211.324773


In [ ]:
X_test=X_test.merge(test_te, left_index=True,right_index=True)

In [ ]:
#Separando os dados para rodar o modelo
X_train_model =X_train.drop(["bairro_group_x","bairro_x" ,"room_type_x"],axis=1)

X_test_model =X_test.drop(["bairro_group_x","bairro_x" ,"room_type_x"],axis=1)


In [ ]:
X_train_model.head()

,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365,bairro_group_y,bairro_y,room_type_y
45708,1,7,2,50,124.577136,103.590747,90.114488
33440,4,11,7,337,118.522388,95.635395,90.114488
21213,3,6,1,0,124.577136,84.869048,90.114488
20048,5,39,1,12,196.847845,90.224049,211.324773
20963,1,10,1,175,99.649351,95.077868,90.114488


# Modelo base

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
modelo=LGBMRegressor()
modelo.fit(X_train_model,y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001153 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 817
[LightGBM] [Info] Number of data points in the train set: 34225, number of used features: 7
[LightGBM] [Info] Start training from score 152.526720


LGBMRegressor()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


pred=modelo.predict(X_test_model)
MSE=mean_squared_error(y_test, pred)
MAE=mean_absolute_error(y_test, pred)

print(f' O MSE é {MSE} ')
print(f' O MAE é {MAE} ')

 O MSE é 46667.21021222049 
 O MAE é 62.27410711787905 


# Modelo com preços até 500 dólares



*   Todo o processo de preparação dos dados será feito de volta pois tem que filtrar os preços desde o ínicio, será criada novas variáveis  



##  Preparação dos dados

In [ ]:
#Filtrando os preços até 500
filtered_df = df[df['price'] <= 500]

In [ ]:
filtered_df

,bairro_group,bairro,room_type,price,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365
0,Manhattan,Midtown,Entire home/apt,225,1,45,2,355
1,Manhattan,Harlem,Private room,150,3,0,1,365
2,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,1,194
3,Manhattan,East Harlem,Entire home/apt,80,10,9,1,0
4,Manhattan,Murray Hill,Entire home/apt,200,3,74,1,129
...,...,...,...,...,...,...,...,...
48889,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,2,9
48890,Brooklyn,Bushwick,Private room,40,4,0,2,36
48891,Manhattan,Harlem,Entire home/apt,115,10,0,1,27
48892,Manhattan,Hell's Kitchen,Shared room,55,1,0,6,2


In [ ]:
X_500 = filtered_df.drop(["price"], axis = 1)
y_500 = filtered_df['price']

In [ ]:
X_500

,bairro_group,bairro,room_type,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365
0,Manhattan,Midtown,Entire home/apt,1,45,2,355
1,Manhattan,Harlem,Private room,3,0,1,365
2,Brooklyn,Clinton Hill,Entire home/apt,1,270,1,194
3,Manhattan,East Harlem,Entire home/apt,10,9,1,0
4,Manhattan,Murray Hill,Entire home/apt,3,74,1,129
...,...,...,...,...,...,...,...
48889,Brooklyn,Bedford-Stuyvesant,Private room,2,0,2,9
48890,Brooklyn,Bushwick,Private room,4,0,2,36
48891,Manhattan,Harlem,Entire home/apt,10,0,1,27
48892,Manhattan,Hell's Kitchen,Shared room,1,0,6,2


In [ ]:
from sklearn.model_selection import train_test_split

X_train_500, X_test_500, y_train_500, y_test_500 = train_test_split(X_500,y_500,test_size=0.30,random_state=30)

In [ ]:
TE_encoder = TargetEncoder()
train_te_500 = TE_encoder.fit_transform(X_train_500[["bairro_group","bairro","room_type"]], y_train_500)
test_te_500 = TE_encoder.transform(X_test_500[["bairro_group","bairro","room_type"]])

train_te_500.head()

,bairro_group,bairro,room_type
33704,163.745097,145.146939,179.529864
10888,112.208165,132.338668,82.066920
35622,112.208165,97.042779,82.066920
38226,163.745097,185.682069,179.529864
35884,163.745097,164.782705,179.529864


In [ ]:
del(X_train_500)

In [ ]:
X_train_500=X_train_500.merge(train_te_500, left_index=True,right_index=True)
X_test_500=X_test_500.merge(test_te_500, left_index=True,right_index=True)

X_train_500

,bairro_group_x,bairro_x,room_type_x,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365,bairro_group_y,bairro_y,room_type_y
33704,Manhattan,Chinatown,Entire home/apt,30,3,11,113,163.745097,145.146939,179.529864
10888,Brooklyn,Williamsburg,Private room,3,1,1,0,112.208165,132.338668,82.066920
35622,Brooklyn,Bedford-Stuyvesant,Private room,2,0,1,88,112.208165,97.042779,82.066920
38226,Manhattan,Nolita,Entire home/apt,3,5,1,0,163.745097,185.682069,179.529864
35884,Manhattan,Upper West Side,Entire home/apt,2,17,1,18,163.745097,164.782705,179.529864
...,...,...,...,...,...,...,...,...,...,...
47220,Brooklyn,Williamsburg,Entire home/apt,2,0,1,14,112.208165,132.338668,179.529864
33885,Manhattan,Upper East Side,Entire home/apt,3,0,1,132,163.745097,162.108855,179.529864
45780,Queens,East Elmhurst,Entire home/apt,1,8,1,49,92.876764,76.489432,179.529864
4649,Brooklyn,Flatbush,Entire home/apt,1,5,1,0,112.208165,85.737443,179.529864


In [ ]:
X_train_model_500 =X_train_500.drop(["bairro_group_x","bairro_x" ,"room_type_x"],axis=1)

X_test_model_500 =X_test_500.drop(["bairro_group_x","bairro_x" ,"room_type_x"],axis=1)

In [ ]:
X_train_model_500

,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365,bairro_group_y,bairro_y,room_type_y
33704,30,3,11,113,163.745097,145.146939,179.529864
10888,3,1,1,0,112.208165,132.338668,82.066920
35622,2,0,1,88,112.208165,97.042779,82.066920
38226,3,5,1,0,163.745097,185.682069,179.529864
35884,2,17,1,18,163.745097,164.782705,179.529864
...,...,...,...,...,...,...,...
47220,2,0,1,14,112.208165,132.338668,179.529864
33885,3,0,1,132,163.745097,162.108855,179.529864
45780,1,8,1,49,92.876764,76.489432,179.529864
4649,1,5,1,0,112.208165,85.737443,179.529864


## Modelo até 500 dólares

In [ ]:
modelo=LGBMRegressor()
modelo.fit(X_train_model_500,y_train_500)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 813
[LightGBM] [Info] Number of data points in the train set: 33495, number of used features: 7
[LightGBM] [Info] Start training from score 131.610330


LGBMRegressor()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


pred=modelo.predict(X_test_model_500)
MSE=mean_squared_error(y_test_500, pred)
MAE=mean_absolute_error(y_test_500, pred)

print(f' O MSE é {MSE} ')
print(f' O MAE é {MAE} ')

 O MSE é 3684.7444919119152 
 O MAE é 40.19029080019619 




*   No modelo base o  O MSE é 46667.21021222049 e o MAE é 62.27410711787905

*   Utilizando preços até 500 dólares melhora MAE e MSE, será utilizado na Tunagem de Hiperparâmetros






# Tunagem de Hiperparâmetros

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params = {
    'max_depth': [3,5,7,9,11,15,20,50,100],
    'n_estimators':[50,100,300,400,600,800],
    'learning_rate': [0.05,0.1,0.2],
    'reg_alpha': [0,0.1,0.5,0.1,0.2],
    'reg_lambda': [0,0.1,0.5],
    "num_leaves" : [20,50,100,200]
}
modelo=LGBMRegressor()
lgbm=RandomizedSearchCV(modelo, param_distributions = params,
                                   scoring = 'neg_mean_absolute_error',
                                   cv = 100,
                                   verbose = 1,
                                   n_iter = 10,
                                   n_jobs = -1)
lgbm.fit(X_train_model_500,y_train_500)

Fitting 100 folds for each of 10 candidates, totalling 1000 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 813
[LightGBM] [Info] Number of data points in the train set: 33495, number of used features: 7
[LightGBM] [Info] Start training from score 131.610330


RandomizedSearchCV(cv=100, estimator=LGBMRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7, 9, 11, 15, 20,
                                                      50, 100],
                                        'n_estimators': [50, 100, 300, 400, 600,
                                                         800],
                                        'num_leaves': [20, 50, 100, 200],
                                        'reg_alpha': [0, 0.1, 0.5, 0.1, 0.2],
                                        'reg_lambda': [0, 0.1, 0.5]},
                   scoring='neg_mean_absolute_error', verbose=1)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


pred=lgbm.predict(X_test_model_500)
MSE=mean_squared_error(y_test_500, pred)
MAE=mean_absolute_error(y_test_500, pred)

print(f' O MSE é {MSE} ')
print(f' O MAE é {MAE} ')

 O MSE é 3668.0684507560663 
 O MAE é 40.09071009962964 


In [ ]:
lgbm.best_params_

{'reg_lambda': 0,
 'reg_alpha': 0.1,
 'num_leaves': 20,
 'n_estimators': 600,
 'max_depth': 15,
 'learning_rate': 0.05}

In [ ]:
lgbm_final=lgbm.best_estimator_

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


pred=lgbm_final.predict(X_test_model_500)
MSE=mean_squared_error(y_test_500, pred)
MAE=mean_absolute_error(y_test_500, pred)

print(f' O MSE é {MSE} ')
print(f' O MAE é {MAE} ')

 O MSE é 3668.0684507560663 
 O MAE é 40.09071009962964 


In [ ]:
lgbm_final

LGBMRegressor(learning_rate=0.05, max_depth=15, n_estimators=600, num_leaves=20,
              reg_alpha=0.1, reg_lambda=0)

In [ ]:
lgbm_final.feature_name_

['minimo_noites',
 'numero_de_reviews',
 'calculado_host_listings_count',
 'disponibilidade_365',
 'bairro_group_y',
 'bairro_y',
 'room_type_y']

# Previsão nos dados selecionado



*   Será feito a previsão com o dados selecionados.
*   Será feito um dataframe só com as colunas necessária para o modelo e será comparada a previsão com o valor real de 225





'id': 2595,

 'nome': 'Skylit Midtown Castle',

'host_id': 2845,

 'host_name': 'Jennifer',

 'bairro_group': 'Manhattan',

 'bairro': 'Midtown',

 'latitude': 40.75362,

 'longitude': -73.98377,
 '

 room_type': 'Entire home/apt',

 'price': 225,

 'minimo_noites': 1,

 'numero_de_reviews': 45,

 'ultima_review': '2019-05-21',

 'reviews_por_mes': 0.38,

 'calculado_host_listings_count': 2,

 'disponibilidade_365': 355}




In [ ]:
colunas=lgbm_final.feature_name_

['minimo_noites',
 'numero_de_reviews',
 'calculado_host_listings_count',
 'disponibilidade_365',
 'bairro_group_y',
 'bairro_y',
 'room_type_y']



*   As colunas que o modelo precisa



In [ ]:
#Selecionando a base para verificar os valores dos bairros e tipo de locação

,bairro_group_x,bairro_x,room_type_x,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365,bairro_group_y,bairro_y,room_type_y
38373,Manhattan,Midtown,Entire home/apt,3,16,1,321,163.745097,219.686335,179.529864




*   bairro_group_y=Manhattan=163.745097

*   bairro_x=bairro_y	=219.686335

*   room_type_x=room_type_y = 179.529864





In [ ]:
filtered_df = df[df['price'] <= 500]

In [ ]:
#Criando o dataframe
ap={'minimo_noites':[1],
 'numero_de_reviews':[45],
 'calculado_host_listings_count':[2],
 'disponibilidade_365':[355],
 'bairro_group_y':[163.745097],
 'bairro_y':[219.686335],
 'room_type_y':[179.529864]}


ap = pd.DataFrame.from_dict(ap)
ap

,minimo_noites,numero_de_reviews,calculado_host_listings_count,disponibilidade_365,bairro_group_y,bairro_y,room_type_y
0,1,45,2,355,163.745097,219.686335,179.529864


In [ ]:
lgbm_final.predict(ap)

array([251.12565236])



*   O modelo previu o valor de 251,12
*   Preço real 225
*   Erro de 26,12 dólares
*   11,60% de erro












# Salvando o modelo escolhido

In [ ]:
import pickle

In [ ]:
pickle.dump(lgbm_final, open('model_final.pkl', 'wb'))

In [ ]:
pickled_model = pickle.load(open('model_final.pkl', 'rb'))
pickled_model.predict(ap)

array([251.12565236])